# Задание по программированию: Линейная алгебра: сходство текстов

Данное задание основано на материалах секции, посвященной введению в линейную алгебру. Вам понадобится компьютер с установленным интерпретатором Python и подключенными библиотеками NumPy и SciPy.

<b>Вы научитесь:</b>
* читать тексты из файла с помощью Python и разбивать их на слова
* переводить тексты в векторные пространства, вычислять расстояния в этих пространствах
* решать системы линейных уравнений
* приближать любые функции с помощью многочленов

<b>Введение</b>

В этом задании вы познакомитесь с некоторыми базовыми методами из линейной алгебры, реализованными в пакете SciPy — в частности, с методами подсчета косинусного расстояния и решения систем линейных уравнений. Обе эти задачи еще много раз встретятся нам в специализации. Так, на решении систем линейных уравнений основана настройка линейных моделей — очень большого и важного класса алгоритмов машинного обучения. Косинусное расстояние же часто используется в анализе текстов для измерения сходства между ними.

<b>Материалы</b>

Справка по функциям пакета scipy.linalg:
http://docs.scipy.org/doc/scipy/reference/linalg.html

Справка по работе с файлами в Python:
https://docs.python.org/2/tutorial/inputoutput.html#reading-and-writing-files

Справка по регулярным выражениям в Python (если вы захотите узнать про них чуть больше):
https://docs.python.org/2/library/re.html

<b>Инструкция по выполнению</b>

Данное задание состоит из двух частей. В каждой ответом будет набор чисел, который вам нужно будет ввести в соответствующее поле через пробел.

<b>Задача 1: сравнение предложений</b>

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

* кошки (животные)
* UNIX-утилита cat для вывода содержимого файлов
* версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

Выполните следующие шаги:

1. Скачайте файл с предложениями (sentences.txt).
2. Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
3. Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
4. Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
5. Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
6. Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
7. Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).
8. Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.



In [1]:
file = open('sentences.txt', 'r')
sentences = []
for line in file.readlines():
    sentences.append(line.lower())
file.close()

In [2]:
import re
import numpy as np

In [3]:
words = np.array(re.split('[^a-z]', " ".join(sentences)))

In [4]:
empty_words_ind = []
for i in range(len(words)):
    if words[i] == '':
        empty_words_ind.append(i)
words = np.delete(words, empty_words_ind)

In [5]:
dif_words = list(set(words))

In [6]:
dictionary = {dif_words[i] : i for i in range(len(dif_words))}

In [7]:
sent_matrix = np.zeros((len(sentences), len(dictionary)))

In [8]:
for i in range(len(sentences)):
    sent = sentences[i]
    words_in_sent = re.split('[^a-z]', sent)
    for word in words_in_sent:
        if word is not '':
            word_ind = dictionary[word]
            sent_matrix[i][word_ind] += 1

<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-8-bdfd78552830>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if word is not '':


In [9]:
import scipy.spatial as sp

In [10]:
first_comp_sent = sent_matrix[0]

In [11]:
distances = []
for i in range(len(sent_matrix)):
    second_comp_sent = sent_matrix[i]
    dist = sp.distance.cosine(first_comp_sent, second_comp_sent)
    distances.append(dist)

In [12]:
first_min_ind = 1
first_min_val = distances[first_min_ind]
second_min_ind = 2
second_min_val = distances[second_min_ind]
for i in range(1, len(distances)):
    if distances[i] <= first_min_val:
        second_min_ind = first_min_ind
        second_min_val = first_min_val
        first_min_ind = i
        first_min_val = distances[i]
    elif distances[i] < second_min_val:
        second_min_ind = i
        second_min_val = distances[i]
if second_min_ind < first_min_val:
    buf = first_min_ind
    first_min_ind = second_min_ind
    second_min_ind = first_min_ind
    buf = first_min_val
    first_min_val = second_min_val
    second_min_val = first_min_val

In [13]:
output_file = open('result_1.txt', 'w')
output_file.write(" ".join([str(first_min_ind), str(second_min_ind)]))
output_file.close()

In [14]:
print(first_min_ind)
print(second_min_ind)
print(first_min_val)
print(second_min_val)
print(distances)

6
4
0.7327387580875756
0.7770887149698589
[0.0, 0.9527544408738466, 0.8644738145642124, 0.8951715163278082, 0.7770887149698589, 0.9402385695332803, 0.7327387580875756, 0.9258750683338899, 0.8842724875284311, 0.9055088817476932, 0.8328165362273942, 0.8804771390665607, 0.8396432548525454, 0.8703592552895671, 0.8740118423302576, 0.9442721787424647, 0.8406361854220809, 0.956644501523794, 0.9442721787424647, 0.8885443574849294, 0.8427572744917122, 0.8250364469440588]
